In [139]:
# 링크에 있는 내용을 한꺼번에 처리하기 (크롤링 & 스크래핑)

# join(base, "상대경로") : 재귀적으로 탐색하기 위한 함수
# 경로주소 처리(경로를 바꿔줌), 문자열로 반환

from urllib.parse import urljoin

base = "https://example.com/html/a.html"
print(urljoin(base, "b.html"))          # 경로 이동 - https://example.com/html/b.html
print(urljoin(base, "sub/c.html"))      # sub/ = ../
print(urljoin(base, "../index.html"))   # https://example.com/index.html
print(urljoin(base, "../img/home.png")) # https://example.com/img/home.png
print(urljoin(base, "../css/home.css")) # https://example.com/css/home.css
print(urljoin(base, "/index.html"))     # https://example.com/index.html
print(urljoin(base, "http://otherExample.com/wiki"))  # http://로 덮어쓰기 - http://otherExample.com/wiki
print(urljoin(base, "//otherExample.com/test"))  # //로 덮어쓰기 - https://otherExample.com/test
                    

https://example.com/html/b.html
https://example.com/html/sub/c.html
https://example.com/index.html
https://example.com/img/home.png
https://example.com/css/home.css
https://example.com/index.html
http://otherExample.com/wiki
https://otherExample.com/test


In [ ]:
# 처리 순서
# 1. HTML 분석
# 2. 링크 추출
# 3. 각 링크 대상에 대해 다음 동작을 구동
    # 3-1. 다운로드 파일인 경우 다운로드하기 (이미지파일, css파일 등..)
    # 3-2. 파일이 HTML이라면 1번으로 돌아가서 다시 실행

In [ ]:
# 모든 페이지를 한번에 다운받는 프로그램 만들기(html..)

# 모듈
from bs4 import BeautifulSoup
from urllib.request import *
from urllib.parse import *
from os import makedirs
import os.path, time, re

# 처리한 내용이 파일인지 아닌지 확인을 위한 변수
proc_files = {}

# HTML 내부에 있는 링크를 추출하는 함수
def enum_links(html, base) :
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.select("link[rel='stylesheet']")   # css 불러오기
    links += soup.select("a[href]")     # href 속성을 가지고 있는 a태그 수집
    
    result = []    
    # href 속성을 추출하고 링크를 절대경로로 변환하여 저장
    for a in links :
        href = a.attrs['href']      # a태그의 href 속성값만 추출
        url = urljoin(base, href)   # base url을 a태그의 href 경로로 변경
        result.append(url)
    return result

# 파일을 다운로드하고 저장하는 함수
def download_file(url) :
    o = urlparse(url)
    savepath = "./"+ o.netloc + o.path
    if re.search(r"/$", savepath) :     # savepath의 마지막이 '/'으로 끝나면 = 디렉토리라면
        savepath += "index.html"        # 링크의 기본문서로 처리하기
    savedir = os.path.dirname(savepath) # savepath의 디렉토리 정보만 빼오기
    
    # 모두 다운로드 되었는지 여부 확인
    if os.path.exists(savepath) : return savepath       # 해당 이름의 파일이 존재한다면 리턴
    # 다운로드가 되지 않았다면, 다운로드 받을 디렉터리 생성
    if not os.path.exists(savedir) :
        print("mkdir =", savedir)
        makedirs(savedir)
    
    # 파일 다운받기
    try :
        print("download =", url)
        urlretrieve(url, savepath)
        time.sleep(3)   # 다운받는 시간 동안 프로그램 일시정지
        return savepath
    except :
        print("다운로드 실패!",url)
        return None
    
# 다운받은 파일이 HTML인지 분석
def analyze_html(url, root_url) :
    savepath = download_file(url)
    if savepath is None : return
    if savepath in proc_files : return    # proc_files에는 처리된 파일명 입력 예정 -> 이미 처리된 파일인 경우 pass
    proc_files[savepath] = True
    print("analyze_html =",url)
    
    # 링크 추출
    html = open(savepath, "r", encoding='utf-8').read()
    links = enum_links(html, url)
    
    for link_url in links :
        # 링크가 루트경로(처음 시작 경로) 이외의 경로를 나타낸다면 무시하기
        if link_url.find(root_url) != 0 :
            if not re.search(r".css$", link_url) : continue
        
        # HTML이라면
        if re.search(r".(html|htm)$", link_url) :
            # 재귀적으로 html 파일 분석
            analyze_html(link_url, root_url)
            continue
        # 기타 파일이라면
        download_file(link_url)

# url에 있는 모든 것 다운받기
url = "https://docs.python.org/3.10/library/"
analyze_html(url, url)




# if __name__ == "__main__" :   # 일반 파이썬 파일인 경우 사용하는 코드
#     # url에 있는 모든 것 다운받기
#     url = "https://docs.python.org/3.10/library/"
#     analyze_html(url, url)
        

In [146]:
# 참고 ! netloc, path 등등 사용법
url = "https://example.com/html/a.html"
t = urlparse(url)
print(t)        # ParseResult(scheme='https', netloc='example.com', path='/html/a.html', params='', query='', fragment='')
print(t.netloc)     # example.com
print(t.path)       # /html/a.html
print(os.path.dirname(url))

ParseResult(scheme='https', netloc='example.com', path='/html/a.html', params='', query='', fragment='')
example.com
/html/a.html
https://example.com/html


In [ ]:
### requests 패키지를 이용한 접근 (pip list를 통해 설치되어 있는지 확인)

# 파이썬으로 로그인 해보기(는 실패,,, 보안이 약한 사이트 어디 없는쥐,,,,)
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# 아이디와 비밀번호 지정
USER = "wntpdls"
PASS = "wntpdls"

# 세션 시작
session = requests.session()
# 로그인
login_info = {
    "m_id" : USER,
    "m_passwd" : PASS
}

url_login = "https://www.hanbit.co.kr/member/login.html"
res = session.post(url_login, data=login_info)
res.raise_for_status()  # 오류 발생 시 예외 처리

# 마이페이지 접근
url_mypage = "https://www.hanbit.co.kr/myhanbit/myhanbit.html"
res = session.get(url_mypage)
res.raise_for_status()


In [36]:
# requests 모듈의 메서드

# get 요청
r = requests.get("http://www.naver.com")

# post 요청
formdata = {"key1":"value1","key2":"value2"}
r = requests.post("http://httpbin.org/",data=formdata)


In [ ]:
# ?????
r = requests.put("https://webhacking.kr/put.txt")      # bytes 데이터를 해당 url로 업로드
print(r.text)
r = requests.delete("http://httpbin.org/delete")
print(r.text)
r = requests.head("https://jsonplaceholder.typicode.com/")     # 헤더 정보만 받아옴(구글은 데이터 X)
print(r.text)


In [50]:
# 테스트 창 열기
from selenium import webdriver
path = "c:\\webdriver\\chromedriver.exe"
driver = webdriver.Chrome(path)

# 한빛 로그인 페이지로 들어오기
url_login = "https://www.hanbit.co.kr/member/login.html"
driver.get(url_login)

# 로그인 창에 ID와 비밀번호 입력
USER = "wntpdls"
PASS = "wntpdls95"

m_id = driver.find_element_by_id('m_id')
m_id.clear()
m_id.send_keys(USER)
m_passwd = driver.find_element_by_id('m_passwd')
m_passwd.clear()
m_passwd.send_keys(PASS)

# 로그인 버튼 클릭
driver.find_element_by_class_name('btn_login').click()

C:\Users\tjoeun709\AppData\Local\Temp\ipykernel_2836\1448183313.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [51]:
# 마이페이지 접속
driver.find_element_by_class_name('myhanbit').click()

#url_mypage = "https://www.hanbit.co.kr/myhanbit/myhanbit.html"
#driver.get(url_mypage)

mileage = driver.find_element_by_css_selector('.mileage_section1 span')
print('한빛 마일리지 :',mileage.text)

coin = driver.find_element_by_css_selector('.mileage_section2 span')
print('한빛 코인 :',coin.text)

In [ ]:
# 네이버 자동 로그인 해보기 - 보안 강해서 실패 ..

driver.get('https://www.naver.com')
driver.find_element_by_class_name('link_login').click()

id = driver.find_element_by_id('id')
id.clear()
id.send_keys('sayin95')

pw = driver.find_element_by_id('pw')
pw.clear()
pw.send_keys('wntpdls145')
driver.find_element_by_class_name('btn_login').click()

now = driver.current_url

if 'nid.naver.com' in now :
    while 1 :
        pw = driver.find_element_by_id('pw')
        pw.clear()
        pw.send_keys('wntpdls145')

        answer = input()
        text = driver.find_element_by_id('captcha')
        text.clear()
        text.send_keys('answer')
        
        driver.find_element_by_class_name('btn_login').click()
        time.sleep(5)
        now = driver.current_url
        if 'www.naver.com' in now : break


driver.get("https://order.pay.naver.com/home")
points = driver.find_element_by_css_selector('.my_npoint strong')
print('네이버페이 포인트 :', points.text)

In [ ]:
# SELinux : 리눅스의 보안 관련 기능 - 세션 정보를 유지시켜줌
#             Enforcing : SELinux를 사용하면 SELinux 정책 규칙에 따라 액세스할 수 있습니다.
#             Permissive : SELinux는 시행 모드에서 실행 중인 경우 거부되었을 작업만 기록합니다.
#             Disabled : 로드된 SELinux 정책이 없습니다.  => 연습용으로 하둡을 가동할 때 꺼줬었음!
            